# L2: Your First Coding Agent 🤖

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> , notebooks and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

First start by filtering warnings and loading environment variables.

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from helper import load_env
load_env()

Try out a demo of the code generation agent below. You can try the prompt shown in the lesson here:
> `Can you create a function for me that draws an emoji and runs it?`

In [ ]:
from demos import coding_agent_demo_ui

coding_agent_demo_ui()

## Language Model

You can start building a coding agent yourself. Start by testing out a generic LLM call to make sure it's working. For simplicity, a function called `llm` is defined for you to handle the boilerplate code for calling the LLM.

In [ ]:
from openai import OpenAI
from llm import llm

client = OpenAI()

messages = [{"role": "user", "content": "hi!"}]
system = "You speak like a linkedin influencer"

response = llm(client, messages, system)

response.output_text

## Tools

You can now create two important pieces of code:
* `Execution`: A class that stores the results of the generated code and errors from your code generation agent.
* `execute_code`: A function that takes any generated code as a string and outputs an `Execution` with the results (of the generated code) and/or any errors.

In [ ]:
from typing import TypedDict
import sys
from io import StringIO


class Execution(TypedDict):
    results: list[str]
    errors: list[str]


def execute_code(code: str) -> Execution:
    execution = {"results": [], "errors": []}
    old_stdout = sys.stdout
    try:
        sys.stdout = StringIO()
        exec(code)
        result = sys.stdout.getvalue()
        sys.stdout = old_stdout
        execution["results"] = [result]
    except Exception as e:
        execution["errors"] = [str(e)]
    finally:
        sys.stdout = old_stdout
        return execution

Test it out by running the cell below.

In [ ]:
execute_code("print('Hello World!')")

You will now define this tool using the schema below and map it to the `tools` dictionary.

In [ ]:
import json

execute_code_schema = {
    "type": "function",
    "name": "execute_code",
    "description": "Execute Python code and return the result or error.",
    "parameters": {
        "type": "object",
        "properties": {
            "code": {
                "type": "string",
                "description": "Python code to execute as a string",
            }
        },
        "required": ["code"],
        "additionalProperties": False,
    },
}

tools = {"execute_code": execute_code}

You will now create a function that executes tools and returns the results. It will take the following arguments:
  * `name`: The name of the function
  * `args`:  The arguments to be passed to the function
  * `tools`: The dictionary with the tool mapping

In [ ]:
from typing import Callable
import json


def execute_tool(name: str, args: str, tools: dict[str, Callable]):
    try:
        args = json.loads(args)
        if name not in tools:
            return {"error": f"Tool {name} doesn't exist."}
        result = tools[name](**args)
    except json.JSONDecodeError as e:
        result = {"error": f"{name} failed to parse arguments: {str(e)}"}
    except KeyError as e:
        result = {"error": f"Missing key in arguments: {str(e)}"}
    except Exception as e:
        result = {"error": str(e)}
    return result

Now you can put the pieces together into the coding agent that will execute code! The coding agent will take the following arguments:
* `client`: The LLM client you will use
* `query`: The user query to the LLM
* `system`: The system prompt for the LLM
* `tools`: The dictionary mapping of the tools
* `tools_schemas`: The tool schemas you defined for the LLM

In [ ]:
def coding_agent(
    client: OpenAI,
    query: str,
    system: str,
    tools: dict[str, Callable],
    tools_schemas: list[dict],
):
    messages = [{"role": "user", "content": query}]
    response = llm(client, messages, system, tools=tools_schemas)
    for part in response.output:
        if part.type == "message":
            print(part.content)
        elif part.type == "function_call":
            name = part.name
            print(f"[{name}] executing...")
            result = execute_tool(name, part.arguments, tools)
            print(f"[{name}] {result}")

In [ ]:
system = """You are a Senior Python programmer.
You must always use the `execute_code` tool to run code.
You collect user's inputs by using the `input` python function.
"""

query = """Make a program that asks how many cups of coffee you had today,
then converts that into lines of code written and prints them back.
Because caffeine = productivity."""

coding_agent(client,
             query,
             system,
             tools=tools,
             tools_schemas=[execute_code_schema])

### File System

Now that you have made your working code agent, you can give it more tools to give it the ability to do even more tasks. You will create two schemas and their associated functions to read and write files.

In addition, you will create a `ToolError` class that will pass an exception back instead of the whole stack trace.

In [ ]:
read_file_schema = {
    "type": "function",
    "name": "read_file",
    "description": "Reads content from a file with optional offset and limit for large files.",
    "parameters": {
        "type": "object",
        "properties": {
            "file_path": {
                "type": "string",
                "description": "Absolute path to the file to read",
            },
            "limit": {
                "type": "number",
                "description": "Maximum number of characters to read",
            },
            "offset": {
                "type": "number",
                "description": "Starting position in the file",
            },
        },
        "required": ["file_path"],
        "additionalProperties": False,
    },
}


In [ ]:
write_file_schema = {
    "type": "function",
    "name": "write_file",
    "description": "Writes content to a file, creating directories if needed.",
    "parameters": {
        "type": "object",
        "properties": {
            "content": {
                "type": "string",
                "description": "Content to write to the file",
            },
            "file_path": {
                "type": "string",
                "description": "Absolute path where the file will be written",
            },
        },
        "required": ["content", "file_path"],
        "additionalProperties": False,
    },
}

In [ ]:
from typing import Optional, Dict, Any

class ToolError(Exception):
    """Custom exception for tool failures"""
    pass

def read_file(
    file_path: str, limit: Optional[int] = None, offset: int = 0
) -> Dict[str, Any]:
    """Read file content with optional offset and limit."""
    if not os.path.exists(file_path):
        raise ToolError(f"File does not exist: {file_path}")

    if not os.path.isfile(file_path):
        raise ToolError(f"Path is not a file: {file_path}")

    try:
        with open(file_path, "r", encoding="utf-8", errors="replace") as f:
            if offset > 0:
                f.seek(offset)
            content = f.read(limit) if limit else f.read()

        return {"content": content, "size": len(content)}

    except PermissionError:
        raise ToolError(f"Permission denied: {file_path}")
    except UnicodeDecodeError:
        raise ToolError(f"Cannot decode file as UTF-8: {file_path}")

In [ ]:
def write_file(content: str, file_path: str) -> Dict[str, Any]:
    """Write content to file, creating directories if needed."""
    try:
        directory = os.path.dirname(file_path)
        if directory:
            os.makedirs(directory, exist_ok=True)

        with open(file_path, "w", encoding="utf-8") as f:
            f.write(content)

        file_size = os.path.getsize(file_path)
        # we keep the result minimal
        return {
            "message": f"Written {file_size} bytes to {file_path}",
            "size": file_size,
        }

    except PermissionError:
        raise ToolError(f"Permission denied: {file_path}")

Now add these tools to the `tools` dictionary.

In [ ]:
tools = {
    "execute_code": execute_code,
    "read_file": read_file,
    "write_file": write_file,
}

Let's try it out! Since you will be working with files on the computer, you will import `os` and create a directory to hold the files the agent creates.

In [ ]:
import os

working_dir = os.getcwd() + "/agent_files"

os.makedirs(working_dir, exist_ok=True)

query = f"""Can you create a blank text.txt file?
Your current working directory is {working_dir}"""

coding_agent(
    client,
    query,
    system,
    tools=tools,
    tools_schemas=[execute_code_schema,
                   read_file_schema,
                   write_file_schema],
)

In [ ]:
query = "Can you read not_exists.txt file and print the content?"

coding_agent(
    client,
    query,
    system,
    tools=tools,
    tools_schemas=[execute_code_schema,
                   read_file_schema,
                   write_file_schema],
)

In [ ]:
query = """Can you:
1. Create a file1.txt file with 'file1'
2. Read file1.txt'
"""

coding_agent(
    client,
    query,
    system,
    tools=tools,
    tools_schemas=[execute_code_schema,
                   read_file_schema,
                   write_file_schema],
)

## Agent Loop

Now it's time to create a loop for your coding agent so it knows when to stop. You will implement two stop conditions:
* When the agent has reached `max_steps`
* When the agent no longer calls functions

By using `max_steps` you help the LLM avoid continually calling functions while not accomplishing its task. And if the agent no longer calls functions, you can assume that it has found the answer and no longer needs to augment its answers.



In [ ]:
import json

def coding_agent(
    client: OpenAI,
    query: str,
    system: str,
    tools: dict[str, Callable],
    tools_schemas: list[dict],
    messages: list[dict] = None,
    max_steps: int = 5,
):
    if messages is None:
        messages = []
    messages.append({"role": "user", "content": query})
    steps = 0

    while steps < max_steps:
        response = llm(client, messages, system, tools=tools_schemas)
        print(f"[#{steps}-step]")
        has_function_call = False

        for part in response.output:
            messages.append(part)
            if part.type == "message":
                print(f"[agent] {response.output_text}")
            elif part.type == "function_call":
                has_function_call = True
                name = part.name
                print(f"[agent][{name}] executing...")
                result = execute_tool(name, part.arguments, tools)
                print(f"[{name}] {result}")
                messages.append(
                    {
                        "type": "function_call_output",
                        "call_id": part.call_id,
                        "output": json.dumps(result),
                    }
                )

        if not has_function_call:
            print("[agent] all tasks completed")
            break
        steps += 1

Try it out by creating a Caesar cipher program.

In [ ]:
query = """Your task is:
1. Write me a caesar cipher function
2. Ask the user for a message
3. Ask the user for the cipher shift
4. Run the function with the user inputs
5. Print back the ciphered message
6. Save it to secret.txt file in the current folder
"""

coding_agent(
    client,
    query,
    system,
    tools=tools,
    tools_schemas=[execute_code_schema,
                   read_file_schema,
                   write_file_schema],
)

In [ ]:
!cat secret.txt

## Chat Interface

You can create a chat interface for your coding agent. This will run code for you. If you would like to exit the chat interface, use `/exit`.

In [ ]:
messages = []

while (query := input(">:")) != "/exit":
    coding_agent(
        client,
        query,
        system,
        messages=messages,
        tools=tools,
        tools_schemas=[execute_code_schema,
                       read_file_schema,
                       write_file_schema],
    )